# Imports 

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import TextVectorization, Embedding, SimpleRNN, Dense, LSTM, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

from sklearn.model_selection import train_test_split
import multiprocessing
from mlflow import MlflowClient
import mlflow
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import sys
from pathlib import Path
cwd = Path.cwd()
parent = cwd.parent
sys.path.append(str(parent))
sys.path.append(os.path.abspath(os.path.join("..","Source")))

mlruns_path = Path("../mlruns").resolve() 
mlflow_uri = mlruns_path.as_uri()
mlflow.set_tracking_uri(mlflow_uri)

from preprocess_data import *  ## import all functions from preprocess_data.py
from postprocess_data import * ## import all functions from postprocess_data.py
from utils import *  ## import all functions from utils.py

# import nltk
# import optuna
# 🤗
from transformers import TFAutoModel, AutoTokenizer



from sklearn.metrics import confusion_matrix 
# from nltk.corpus import stopwords  
from tqdm import tqdm
tqdm.pandas()
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

nw = multiprocessing.cpu_count()



client = MlflowClient(tracking_uri="http://localhost:8080")
os.environ["TF_KERAS"]='1'
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("GPUs disponibles :", tf.config.list_physical_devices("GPU"))
print("Version TF :", tf.__version__)

c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version
c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.10.1
Num GPUs Available:  1
GPUs disponibles : [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Version TF : 2.10.1


# Préparation data

## Importation dataset

In [2]:
df = pd.read_csv('https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+7%C2%A0-+D%C3%A9tectez+les+Bad+Buzz+gr%C3%A2ce+au+Deep+Learning/sentiment140.zip',
                header=None,
                compression='zip',
                encoding='cp1252')

df.columns = ['target', 'ids', 'date', 'flag', 'user', 'text']
data_size = 0.02
sample_df, _ = train_test_split(df, test_size=1-data_size, random_state=42, stratify=df['target'])
sample_df = sample_df.reset_index(drop=True)
print(f"Sample size: {sample_df.shape[0]} rows")
data_numrows = sample_df.shape[0]
# On ne garde que les colonnes 'target' et 'text'
sample_df = sample_df[['target', 'text']]
sample_df["target"] = sample_df["target"].apply(lambda x: 0 if x == 0 else 1)
sample_df.to_csv('../Data/raw_data.csv', index=False)
dataset = mlflow.data.from_pandas(
    sample_df,
    source="../Data/raw_data.csv",
    name="dataset_v1"
)


Sample size: 32000 rows


c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\data\dataset_source_registry.py:148: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for '../Data/raw_data.csv'. Exception: 
  return _dataset_source_registry.resolve(
c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\data\dataset_source_registry.py:148: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(


## Train/Validation split

In [3]:
# Data
X_raw = sample_df['text']
y = sample_df['target']
X_train, X_val, y_train, y_val = train_test_split(X_raw, y, test_size=0.2, random_state=42, stratify=y)


# Modèles BERT classiques

## Fonction centrale 

In [5]:
rnn_size = 128
epochs = 100
lr = 1e-3


def test_bert_model(bert_model_name):
    with mlflow.start_run():
        mlflow.log_input(dataset)
        mlflow.log_params(params={
            'rnn_size': rnn_size, 
            'epochs': epochs, 
            'learning_rate': lr,
            'bert_model_name':bert_model_name
        })

        tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
        encodings_train = tokenizer(X_train.to_list(), 
                                    truncation=True, 
                                    padding=True, 
                                    max_length=64,
                                    return_tensors="tf")
        encodings_val = tokenizer(X_val.to_list(), 
                                  truncation=True, 
                                  padding=True, 
                                  max_length = 64,
                                  return_tensors="tf")

        dataset_train = tf.data.Dataset.from_tensor_slices(
            (
                {"input_ids": encodings_train["input_ids"], 
                 "attention_mask": encodings_train["attention_mask"]
                 },y_train
                )
                ).batch(32)
        
        dataset_val = tf.data.Dataset.from_tensor_slices(
            (
                {"input_ids": encodings_val["input_ids"], 
                 "attention_mask": encodings_val["attention_mask"]
                 },y_val
                )
                ).batch(32)
        # On charge le modèle pré-entrainné
        base_model = TFAutoModel.from_pretrained(bert_model_name, from_pt=True)
        base_model.trainable = False # Pas de fine-tuning ou d'entrainement car impossible à faire avec les ressources disponibles

        # Construction du modèle keras 
        ## Une input layer pour les input ids
        input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name="input_ids")
        ## Une input layer pour le masque d'attention
        attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name="attention_mask")
        ## On récupère 

        outputs = base_model(input_ids, attention_mask=attention_mask)
        token_embeddings = outputs.last_hidden_state  # [batch, seq_len, hidden_size]

        # On prend le token [CLS] comme vecteur de phrase
        cls_token = token_embeddings[:, 0, :]
        max_tokens = tf.reduce_max(token_embeddings, axis=1)  #
        mean_tokens = tf.reduce_mean(token_embeddings, axis=1)  #

        all_tokens = tf.concat([cls_token, max_tokens, mean_tokens], axis=-1)  # [batch, hidden_size*2]
    
        x = tf.keras.layers.Dense(rnn_size, activation="relu", 
                                  kernel_regularizer=regularizers.L2(1e-4),
                                  bias_regularizer=regularizers.L2(1e-4), 
                                #   activity_regularizer=regularizers.L2(1e-4),
                                  )(all_tokens)
        x = tf.keras.layers.Dropout(0.5)(x)
        logits = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    
        model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=logits)

        ## Callbacks
        model_savepath = f"./Models/MY_{'_'.join(bert_model_name.split('/'))}_dense{rnn_size}.h5"
        checkpoint = ModelCheckpoint(model_savepath, monitor='val_accuracy', verbose=0, save_best_only=True, save_weights_only=True, mode='max')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=20)
        lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, min_lr=1e-5)
        callbacks_list = [checkpoint, es, lr_scheduler]


        optimizer = Adam(learning_rate=lr)
        model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
        # Summary
        model.summary()
        # History
        with tf.device("/GPU:0"):
            history = model.fit(dataset_train, epochs=epochs, batch_size=64, validation_data=dataset_val, callbacks=callbacks_list, verbose=1)


        model.load_weights(model_savepath)

                # Prédictions sur le jeu de validation
        y_pred_proba = model.predict(dataset_val)
        y_pred = (y_pred_proba>0.5)


        output_dict = postprocess_model_output(y_val, y_pred, y_pred_proba) # voir postprocess_data.py

        # Logging des métriques dans MLflow
        mlflow.log_metrics(output_dict)
        # Matrice de confusion
        cm = confusion_matrix(y_val, y_pred, normalize='pred')
        fig, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues", ax=ax, )
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix - Validation Set")
        fig.savefig("confusion_matrix.png")
        plt.close(fig)
        mlflow.log_artifact("confusion_matrix.png")
        #
        fig2 = plot_training_history(history,show=False)
        fig2.savefig("learning_path.png")
        plt.close(fig2)
        mlflow.log_artifact("learning_path.png")

        # Enregistrement du modèle dans MLflow
        mlflow.tensorflow.log_model(model, "model")
    




## Experiment MLFlow

In [6]:
# Création de l'étude Optuna et optimisation
print("Setting up MLflow experiment...")
mlflow.set_experiment("BERT_models_experiment")
exp_id = mlflow.get_experiment_by_name("BERT_models_experiment").experiment_id

experiment_description = (
    "Comparaison de plusieurs modèles BERT"
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "Sentiment analysis modelling",
    "model_type": "BERT_pretrained",
    "team": "Ph. Constant",
    "project_quarter": "Q3-2025",
    "mlflow.note.content": experiment_description,
}

for key, value in experiment_tags.items():
    client.set_experiment_tag(exp_id, key, value)

Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 367, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 465, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1635, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1628, in _read_helper
    result = read_yaml(root, file_name)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\utils\yaml_utils.py", line 107, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' d

Setting up MLflow experiment...


## Liste des candidats

In [ ]:
bert_model_list = [
    "bert-base-uncased",
    "bert-base-cased",
    "distilbert-base-uncased",
    "distilbert-base-cased",
    "bert-base-multilingual-uncased",
    "bert-base-multilingual-cased",
    "distilroberta-base",
    "xlm-roberta-base"
]



## Lancement experiment

In [8]:
for model_name in bert_model_list:
    print(f"Running test with BERT model : {model_name}")
    test_bert_model(model_name)

800/800 [==============================] - 68s 85ms/step - loss: 0.6120 - accuracy: 0.6503 - val_loss: 0.5641 - val_accuracy: 0.7522 - lr: 0.0010
Epoch 11/100
800/800 [==============================] - 65s 82ms/step - loss: 0.6136 - accuracy: 0.6568 - val_loss: 0.5765 - val_accuracy: 0.7514 - lr: 0.0010
Epoch 12/100
800/800 [==============================] - 66s 83ms/step - loss: 0.6035 - accuracy: 0.6583 - val_loss: 0.5728 - val_accuracy: 0.7402 - lr: 0.0010
Epoch 13/100
800/800 [==============================] - 67s 84ms/step - loss: 0.6046 - accuracy: 0.6568 - val_loss: 0.5642 - val_accuracy: 0.7381 - lr: 0.0010
Epoch 14/100
800/800 [==============================] - 78s 98ms/step - loss: 0.6038 - accuracy: 0.6547 - val_loss: 0.5571 - val_accuracy: 0.7423 - lr: 0.0010
Epoch 15/100
800/800 [==============================] - 78s 97ms/step - loss: 0.6044 - accuracy: 0.6566 - val_loss: 0.5556 - val_accuracy: 0.7544 - lr: 0.0010
Epoch 16/100
800/800 [==============================] - 77s

2025/10/07 13:09:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/07 13:09:48 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmplo4ggctn\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmplo4ggctn\model\data\model\assets
2025/10/07 13:11:11 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\bassm\AppData\Local\Temp\tmplo4ggctn\model, flavor: tensorflow). Fall back to return ['tensorflow==2.10.1']. Set logging level to DEBUG to see the full traceback. 
2025/10/07 13:11:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


# Modèles BERT spécifiques (HuggingFace)

## Fonction centrale 

In [15]:
rnn_size = 128
epochs = 100
lr = 1e-3


def test_bert_model_other(bert_model_name):
    with mlflow.start_run():
        mlflow.log_input(dataset)
        mlflow.log_params(params={
            'rnn_size': rnn_size, 
            'epochs': epochs, 
            'learning_rate': lr,
            'bert_model_name':bert_model_name
        })

        tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
        encodings_train = tokenizer(X_train.to_list(), 
                                    truncation=True, 
                                    padding=True, 
                                    max_length=64,
                                    return_tensors="tf")
        encodings_val = tokenizer(X_val.to_list(), 
                                  truncation=True, 
                                  padding=True, 
                                  max_length = 64,
                                  return_tensors="tf")

        dataset_train = tf.data.Dataset.from_tensor_slices(
            (
                {"input_ids": encodings_train["input_ids"], 
                 "attention_mask": encodings_train["attention_mask"]
                 },y_train
                )
                ).batch(32)
        
        dataset_val = tf.data.Dataset.from_tensor_slices(
            (
                {"input_ids": encodings_val["input_ids"], 
                 "attention_mask": encodings_val["attention_mask"]
                 },y_val
                )
                ).batch(32)
        # On charge le modèle pré-entrainné
        try:
            base_model = TFAutoModel.from_pretrained(bert_model_name, from_pt=False)# Forcer la version tensorflow
            mlflow.log_param("from_pt", False)
        except:
            print(bert_model_name, " not available in TF")
            base_model = TFAutoModel.from_pretrained(bert_model_name,from_pt=True)
            mlflow.log_param("from_pt", True)

            
    
        base_model.trainable = False # Pas de fine-tuning ou d'entrainement car impossible à faire avec les ressources disponibles

        # Construction du modèle keras 
        ## Une input layer pour les input ids
        input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name="input_ids")
        ## Une input layer pour le masque d'attention
        attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name="attention_mask")
        ## On récupère 

        outputs = base_model(input_ids, attention_mask=attention_mask)
        token_embeddings = outputs.last_hidden_state  # [batch, seq_len, hidden_size]

        # On prend le token [CLS] comme vecteur de phrase
        cls_token = token_embeddings[:, 0, :]
        max_tokens = tf.reduce_max(token_embeddings, axis=1)  #
        mean_tokens = tf.reduce_mean(token_embeddings, axis=1)  #

        all_tokens = tf.concat([cls_token, max_tokens, mean_tokens], axis=-1)  # [batch, hidden_size*2]
    
        x = tf.keras.layers.Dense(rnn_size, activation="relu", 
                                  kernel_regularizer=regularizers.L2(1e-4),
                                  bias_regularizer=regularizers.L2(1e-4), 
                                #   activity_regularizer=regularizers.L2(1e-4),
                                  )(all_tokens)
        x = tf.keras.layers.Dropout(0.5)(x)
        logits = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    
        model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=logits)

        ## Callbacks
        model_savepath = f"./Models/MY_{'_'.join(bert_model_name.split('/'))}_dense{rnn_size}.h5"
        checkpoint = ModelCheckpoint(model_savepath, monitor='val_accuracy', verbose=0, save_best_only=True, save_weights_only=True, mode='max')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=20)
        lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, min_lr=1e-5)
        callbacks_list = [checkpoint, es, lr_scheduler]


        optimizer = Adam(learning_rate=lr)
        model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
        # Summary
        model.summary()
        # History
        with tf.device("/GPU:0"):
            history = model.fit(dataset_train, epochs=epochs, batch_size=64, validation_data=dataset_val, callbacks=callbacks_list, verbose=1)


        model.load_weights(model_savepath)

                # Prédictions sur le jeu de validation
        y_pred_proba = model.predict(dataset_val)
        y_pred = (y_pred_proba>0.5)


        output_dict = postprocess_model_output(y_val, y_pred, y_pred_proba) # voir postprocess_data.py

        # Logging des métriques dans MLflow
        mlflow.log_metrics(output_dict)
        # Matrice de confusion
        cm = confusion_matrix(y_val, y_pred, normalize='pred')
        fig, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues", ax=ax, )
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix - Validation Set")
        fig.savefig("confusion_matrix.png")
        plt.close(fig)
        mlflow.log_artifact("confusion_matrix.png")
        #
        fig2 = plot_training_history(history,show=False)
        fig2.savefig("learning_path.png")
        plt.close(fig2)
        mlflow.log_artifact("learning_path.png")

        # Enregistrement du modèle dans MLflow
        mlflow.tensorflow.log_model(model, "model")
    




## Experiment MLFlow

In [16]:
# Création de l'étude Optuna et optimisation
print("Setting up MLflow experiment...")
mlflow.set_experiment("BERT_models_experiment")
exp_id = mlflow.get_experiment_by_name("BERT_models_experiment").experiment_id

experiment_description = (
    "Comparaison de plusieurs modèles BERT"
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "Sentiment analysis modelling",
    "model_type": "BERT_pretrained_huggingface",
    "team": "Ph. Constant",
    "project_quarter": "Q3-2025",
    "mlflow.note.content": experiment_description,
}

for key, value in experiment_tags.items():
    client.set_experiment_tag(exp_id, key, value)

Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 367, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 465, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1635, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1628, in _read_helper
    result = read_yaml(root, file_name)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\utils\yaml_utils.py", line 107, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' d

Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 367, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 465, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1635, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1628, in _read_helper
    result = read_yaml(root, file_name)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\utils\yaml_utils.py", line 107, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' d

Setting up MLflow experiment...


## Liste des candidats

In [ ]:
# D'autrs modèles BERT à tester (spécialisés en analyse de sentiments)
other_bert_models = [
    "distilbert/distilbert-base-uncased-finetuned-sst-2-english",
    "bhadresh-savani/roberta-base-emotion",
    "gilf/english-yelp-sentiment",
    "joeddav/distilbert-base-uncased-go-emotions-student",
    "lordtt13/emo-mobilebert",
    "Kapiche/twitter-roberta-base-sentiment",
    "HARSHU550/Sentiments",
]


## Lancement experiment

In [20]:
for model_name in other_bert_models:
    print(f"Running test with BERT model : {model_name}")
    test_bert_model_other(model_name)

800/800 [==============================] - 142s 178ms/step - loss: 0.5377 - accuracy: 0.7421 - val_loss: 0.5030 - val_accuracy: 0.7638 - lr: 5.0000e-04
Epoch 50/100
800/800 [==============================] - 132s 166ms/step - loss: 0.5315 - accuracy: 0.7456 - val_loss: 0.5140 - val_accuracy: 0.7550 - lr: 5.0000e-04
Epoch 51/100
800/800 [==============================] - 136s 170ms/step - loss: 0.5349 - accuracy: 0.7427 - val_loss: 0.5085 - val_accuracy: 0.7597 - lr: 5.0000e-04
Epoch 52/100
800/800 [==============================] - 136s 170ms/step - loss: 0.5332 - accuracy: 0.7484 - val_loss: 0.5059 - val_accuracy: 0.7613 - lr: 5.0000e-04
Epoch 53/100
800/800 [==============================] - 135s 168ms/step - loss: 0.5308 - accuracy: 0.7479 - val_loss: 0.5066 - val_accuracy: 0.7625 - lr: 5.0000e-04
Epoch 54/100
800/800 [==============================] - 144s 180ms/step - loss: 0.5321 - accuracy: 0.7460 - val_loss: 0.5034 - val_accuracy: 0.7645 - lr: 5.0000e-04
Epoch 55/100
800/800 [=

2025/10/08 01:19:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/08 01:19:50 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpzfjfwu0i\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpzfjfwu0i\model\data\model\assets
2025/10/08 01:21:05 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\bassm\AppData\Local\Temp\tmpzfjfwu0i\model, flavor: tensorflow). Fall back to return ['tensorflow==2.10.1']. Set logging level to DEBUG to see the full traceback. 
2025/10/08 01:21:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Running test with BERT model : joeddav/distilbert-base-uncased-go-emotions-student


...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['pre_classifier.weight', 'classifier.bias', 'classifier.weight', 'pre_classifier.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the m

Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 tf_distil_bert_model_7 (TFDist  TFBaseModelOutput(l  66362880   ['input_ids[0][0]',              
 ilBertModel)                   ast_hidden_state=(N               'attention_mask[0][0]']         
                                one, None, 768),                                                  
                                 hidden_states=None                                        

2025/10/08 08:15:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/08 08:15:38 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpl6zaojzq\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpl6zaojzq\model\data\model\assets
2025/10/08 08:16:08 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\bassm\AppData\Local\Temp\tmpl6zaojzq\model, flavor: tensorflow). Fall back to return ['tensorflow==2.10.1']. Set logging level to DEBUG to see the full traceback. 
2025/10/08 08:16:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


# Optimisation de la tête du modèle 

## Fonction de base

In [9]:
rnn_size = 128
epochs = 100
lr = 1e-3


def test_bert_model_other_v2(bert_model_name, rnn_size=rnn_size):
    with mlflow.start_run():
        mlflow.log_input(dataset)
        mlflow.log_params(params={
            'rnn_size': rnn_size, 
            'epochs': epochs, 
            'learning_rate': lr,
            'bert_model_name':bert_model_name
        })

        tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
        encodings_train = tokenizer(X_train.to_list(), 
                                    truncation=True, 
                                    padding=True, 
                                    max_length=64,
                                    return_tensors="tf")
        encodings_val = tokenizer(X_val.to_list(), 
                                  truncation=True, 
                                  padding=True, 
                                  max_length = 64,
                                  return_tensors="tf")

        dataset_train = tf.data.Dataset.from_tensor_slices(
            (
                {"input_ids": encodings_train["input_ids"], 
                 "attention_mask": encodings_train["attention_mask"]
                 },y_train
                )
                ).batch(32)
        
        dataset_val = tf.data.Dataset.from_tensor_slices(
            (
                {"input_ids": encodings_val["input_ids"], 
                 "attention_mask": encodings_val["attention_mask"]
                 },y_val
                )
                ).batch(32)
        # On charge le modèle pré-entrainné
        try:
            base_model = TFAutoModel.from_pretrained(bert_model_name, from_pt=False)# Forcer la version tensorflow
            mlflow.log_param("from_pt", False)
        except:
            print(bert_model_name, " not available in TF")
            base_model = TFAutoModel.from_pretrained(bert_model_name,from_pt=True)
            mlflow.log_param("from_pt", True)

            
    
        base_model.trainable = False # Pas de fine-tuning ou d'entrainement car impossible à faire avec les ressources disponibles

        # Construction du modèle keras 
        ## Une input layer pour les input ids
        input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name="input_ids")
        ## Une input layer pour le masque d'attention
        attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name="attention_mask")
        ## On récupère 

        outputs = base_model(input_ids, attention_mask=attention_mask)
        token_embeddings = outputs.last_hidden_state  # [batch, seq_len, hidden_size]

        # On prend le token [CLS] comme vecteur de phrase
        cls_token = token_embeddings[:, 0, :]
        max_tokens = tf.reduce_max(token_embeddings, axis=1)  #
        mean_tokens = tf.reduce_mean(token_embeddings, axis=1)  #

        all_tokens = tf.concat([cls_token, max_tokens, mean_tokens], axis=-1)  # [batch, hidden_size*2]
    
        x = tf.keras.layers.Dense(rnn_size, activation="relu", 
                                  kernel_regularizer=regularizers.L2(1e-4),
                                  bias_regularizer=regularizers.L2(1e-4), 
                                #   activity_regularizer=regularizers.L2(1e-4),
                                  )(all_tokens)
        x = tf.keras.layers.Dropout(0.5)(x)
        logits = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    
        model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=logits)

        ## Callbacks
        model_savepath = f"./Models/MY_{'_'.join(bert_model_name.split('/'))}_dense{rnn_size}.h5"
        checkpoint = ModelCheckpoint(model_savepath, monitor='val_accuracy', verbose=0, save_best_only=True, save_weights_only=True, mode='max')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=20)
        lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, min_lr=1e-5)
        callbacks_list = [checkpoint, es, lr_scheduler]


        optimizer = Adam(learning_rate=lr)
        model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
        # Summary
        model.summary()
        # History
        with tf.device("/GPU:0"):
            history = model.fit(dataset_train, epochs=epochs, batch_size=64, validation_data=dataset_val, callbacks=callbacks_list, verbose=1)


        model.load_weights(model_savepath)

                # Prédictions sur le jeu de validation
        y_pred_proba = model.predict(dataset_val)
        y_pred = (y_pred_proba>0.5)


        output_dict = postprocess_model_output(y_val, y_pred, y_pred_proba) # voir postprocess_data.py

        # Logging des métriques dans MLflow
        mlflow.log_metrics(output_dict)
        # Matrice de confusion
        cm = confusion_matrix(y_val, y_pred, normalize='pred')
        fig, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues", ax=ax, )
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix - Validation Set")
        fig.savefig("confusion_matrix.png")
        plt.close(fig)
        mlflow.log_artifact("confusion_matrix.png")
        #
        fig2 = plot_training_history(history,show=False)
        fig2.savefig("learning_path.png")
        plt.close(fig2)
        mlflow.log_artifact("learning_path.png")

        # Enregistrement du modèle dans MLflow
        mlflow.tensorflow.log_model(model, "model")
    




In [5]:
# Création de l'étude Optuna et optimisation
print("Setting up MLflow experiment...")
mlflow.set_experiment("BERT_models_experiment")
exp_id = mlflow.get_experiment_by_name("BERT_models_experiment").experiment_id

experiment_description = (
    "Optimisation de la tête du modèle basé sur BERT."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "Sentiment analysis modelling",
    "model_type": "BERT_pretrained",
    "team": "Ph. Constant",
    "project_quarter": "Q3-2025",
    "mlflow.note.content": experiment_description,
}

for key, value in experiment_tags.items():
    client.set_experiment_tag(exp_id, key, value)

Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 367, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 465, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1635, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1628, in _read_helper
    result = read_yaml(root, file_name)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\utils\yaml_utils.py", line 107, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' d

Setting up MLflow experiment...


In [16]:
tf.keras.backend.clear_session()

In [10]:
for rnn_size in [64, 192, 256]:
    tf.keras.backend.clear_session()
    print(f"Running test with BERT model : Kapiche/twitter-roberta-base-sentiment")
    test_bert_model_other_v2(bert_model_name="Kapiche/twitter-roberta-base-sentiment", rnn_size=rnn_size)

Running test with BERT model : Kapiche/twitter-roberta-base-sentiment
Kapiche/twitter-roberta-base-sentiment  not available in TF


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 tf_roberta_model_1 (TFRobertaM  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 odel)                          thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, None                                           

2025/10/09 00:18:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/09 00:18:13 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmp1ab9pma9\model\data\model\assets


...

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 tf_roberta_model_1 (TFRobertaM  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 odel)                          thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, None                                           

2025/10/09 06:09:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/09 06:09:57 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmp3y_ic57_\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmp3y_ic57_\model\data\model\assets
2025/10/09 06:10:47 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\bassm\AppData\Local\Temp\tmp3y_ic57_\model, flavor: tensorflow). Fall back to return ['tensorflow==2.10.1']. Set logging level to DEBUG to see the full traceback. 
2025/10/09 06:10:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


# Last


In [14]:
rnn_size = 256
epochs = 100
lr = 1e-3


def test_bert_model_other_v2(bert_model_name, rnn_size=rnn_size):
    with mlflow.start_run():
        mlflow.log_input(dataset)
        mlflow.log_params(params={
            'rnn_size': rnn_size, 
            'epochs': epochs, 
            'learning_rate': lr,
            'bert_model_name':bert_model_name
        })

        tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
        encodings_train = tokenizer(X_train.to_list(), 
                                    truncation=True, 
                                    padding=True, 
                                    max_length=64,
                                    return_tensors="tf")
        encodings_val = tokenizer(X_val.to_list(), 
                                  truncation=True, 
                                  padding=True, 
                                  max_length=64,
                                  return_tensors="tf")

        dataset_train = tf.data.Dataset.from_tensor_slices(
            (
                {"input_ids": encodings_train["input_ids"], 
                 "attention_mask": encodings_train["attention_mask"]
                 },y_train
                )
                ).batch(32)
        
        dataset_val = tf.data.Dataset.from_tensor_slices(
            (
                {"input_ids": encodings_val["input_ids"], 
                 "attention_mask": encodings_val["attention_mask"]
                 },y_val
                )
                ).batch(32)
        # On charge le modèle pré-entrainné
        try:
            base_model = TFAutoModel.from_pretrained(bert_model_name, from_pt=False)# Forcer la version tensorflow
            mlflow.log_param("from_pt", False)
        except:
            print(bert_model_name, " not available in TF")
            base_model = TFAutoModel.from_pretrained(bert_model_name,from_pt=True)
            mlflow.log_param("from_pt", True)

            
    
        base_model.trainable = False # Pas de fine-tuning ou d'entrainement car impossible à faire avec les ressources disponibles

        # Construction du modèle keras 
        ## Une input layer pour les input ids
        input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name="input_ids")
        ## Une input layer pour le masque d'attention
        attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name="attention_mask")
        ## On récupère 

        outputs = base_model(input_ids, attention_mask=attention_mask)
        token_embeddings = outputs.last_hidden_state  # [batch, seq_len, hidden_size]

        # On prend le token [CLS] comme vecteur de phrase
        cls_token = token_embeddings[:, 0, :]
        max_tokens = tf.reduce_max(token_embeddings, axis=1)  #
        mean_tokens = tf.reduce_mean(token_embeddings, axis=1)  #

        all_tokens = tf.concat([cls_token, max_tokens, mean_tokens], axis=-1)  # [batch, hidden_size*2]
    
        x = tf.keras.layers.Dense(rnn_size, activation="relu", 
                                  kernel_regularizer=regularizers.L2(1e-4),
                                  bias_regularizer=regularizers.L2(1e-4), 
                                #   activity_regularizer=regularizers.L2(1e-4),
                                  )(all_tokens)
        x = tf.keras.layers.Dropout(0.5)(x)
        logits = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    
        model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=logits)

        ## Callbacks
        model_savepath = f"./Models/MY_{'_'.join(bert_model_name.split('/'))}_dense{rnn_size}.h5"
        checkpoint = ModelCheckpoint(model_savepath, monitor='val_accuracy', verbose=0, save_best_only=True, save_weights_only=True, mode='max')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=20)
        lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, min_lr=1e-5)
        callbacks_list = [checkpoint, es, lr_scheduler]


        optimizer = Adam(learning_rate=lr)
        model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
        # Summary
        model.summary()
        # History
        with tf.device("/GPU:0"):
            history = model.fit(dataset_train, epochs=epochs, batch_size=64, validation_data=dataset_val, callbacks=callbacks_list, verbose=1)


        model.load_weights(model_savepath)

                # Prédictions sur le jeu de validation
        y_pred_proba = model.predict(dataset_val)
        y_pred = (y_pred_proba>0.5)


        output_dict = postprocess_model_output(y_val, y_pred, y_pred_proba) # voir postprocess_data.py

        # Logging des métriques dans MLflow
        mlflow.log_metrics(output_dict)
        # Matrice de confusion
        cm = confusion_matrix(y_val, y_pred, normalize='pred')
        fig, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues", ax=ax, )
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix - Validation Set")
        fig.savefig("confusion_matrix.png")
        plt.close(fig)
        mlflow.log_artifact("confusion_matrix.png")
        #
        fig2 = plot_training_history(history,show=False)
        fig2.savefig("learning_path.png")
        plt.close(fig2)
        mlflow.log_artifact("learning_path.png")

        # Enregistrement du modèle dans MLflow
        mlflow.tensorflow.log_model(model, "model")
    
    return model, tokenizer



In [15]:

print(f"Running test with BERT model Kapiche/twitter-roberta-base-sentiment")
model, tokenizer  = test_bert_model_other_v2(bert_model_name="Kapiche/twitter-roberta-base-sentiment", rnn_size=256)

Running test with BERT model Kapiche/twitter-roberta-base-sentiment
Kapiche/twitter-roberta-base-sentiment  not available in TF


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 tf_roberta_model_6 (TFRobertaM  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 odel)                          thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, None                                         

2025/10/09 20:34:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/09 20:34:48 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpz06ietb2\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpz06ietb2\model\data\model\assets
2025/10/09 20:35:48 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\bassm\AppData\Local\Temp\tmpz06ietb2\model, flavor: tensorflow). Fall back to return ['tensorflow==2.10.1']. Set logging level to DEBUG to see the full traceback. 
2025/10/09 20:35:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [16]:
model


In [ ]:
model.save("../Models/Best_BERT_model")

INFO:tensorflow:Assets written to: ../Models/Best_BERT_model\assets


INFO:tensorflow:Assets written to: ../Models/Best_BERT_model\assets


# Sauvegarde du modèle 

In [19]:

# On définit une fonction de service avec les bons inputs
@tf.function(input_signature=[
    {
        "input_ids": tf.TensorSpec([None, None], tf.int32, name="input_ids"),
        "attention_mask": tf.TensorSpec([None, None], tf.int32, name="attention_mask"),
    }
])
def serving_fn(inputs):
    return model(inputs)

# Sauvegarde au format TensorFlow
tf.saved_model.save(model, "../Models/Model_BERT_save_with_tf2", signatures={"serving_default": serving_fn})


INFO:tensorflow:Assets written to: ../Models/Model_BERT_save_with_tf2\assets


INFO:tensorflow:Assets written to: ../Models/Model_BERT_save_with_tf2\assets


In [20]:
model.save("../Models/Best_BERT_model_save_keras")


INFO:tensorflow:Assets written to: ../Models/Best_BERT_model_save_keras\assets


INFO:tensorflow:Assets written to: ../Models/Best_BERT_model_save_keras\assets


In [22]:
tf.keras.models.save_model(model, "../Models/Best_BERT_model_save_keras_v2", save_format="tf")

INFO:tensorflow:Assets written to: ../Models/Best_BERT_model_save_keras_v2\assets


INFO:tensorflow:Assets written to: ../Models/Best_BERT_model_save_keras_v2\assets


In [23]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 tf_roberta_model_6 (TFRobertaM  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 odel)                          thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, None                                         

# Sauvegarde du tokenizer

In [21]:
tokenizer.save_pretrained("../Models/tokenizer_BERT_model_v1")


('../Models/tokenizer_BERT_model_v1\\tokenizer_config.json',
 '../Models/tokenizer_BERT_model_v1\\special_tokens_map.json',
 '../Models/tokenizer_BERT_model_v1\\vocab.json',
 '../Models/tokenizer_BERT_model_v1\\merges.txt',
 '../Models/tokenizer_BERT_model_v1\\added_tokens.json',
 '../Models/tokenizer_BERT_model_v1\\tokenizer.json')

# Chargement du modèle 

In [ ]:
loaded = tf.saved_model.load("../Models/Best_BERT_model2")
infer = loaded.signatures["serving_default"]



# Chargement du tokenizer

In [ ]:
tokenizer_loaded = AutoTokenizer.from_pretrained("../Models/Best_BERT_tokenizer")


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


# Texte de test 

In [ ]:
text = ["it wasn't so bad actually  ", "bad", "average"]

test_encoding = tokenizer_loaded(text, truncation=True, padding=True, return_tensors="tf")



In [ ]:
test_output = infer(**{
    "input_ids": test_encoding["input_ids"],
    "attention_mask": test_encoding["attention_mask"],
    "token_type_ids": test_encoding.get("token_type_ids", tf.zeros_like(test_encoding["input_ids"]))
})
print(test_output)

{'output_0': <tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[0.5667334 ],
       [0.01679735],
       [0.1408657 ]], dtype=float32)>}


In [ ]:
type(test_output['output_0'].numpy()[0][0])

numpy.float32

In [ ]:
sys.version

'3.10.18 | packaged by conda-forge | (main, Jun  4 2025, 14:42:04) [MSC v.1943 64 bit (AMD64)]'

In [ ]:
os.path.join("..","Models","Best_BERT_model2")

'..\\Models\\Best_BERT_model2'

In [ ]:
dataset_train

<BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
for k in encodings_train.keys():
    print(k)

input_ids
token_type_ids
attention_mask


# Conversion tflite

In [ ]:
import tensorflow as tf

# Charger le modèle keras
model = tf.keras.models.load_model("../Models/Best_BERT_model2", compile=False)

# Convertir vers TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Sauvegarde
with open("bert_model.tflite", "wb") as f:
    f.write(tflite_model)


ValueError: Unable to create a Keras model from SavedModel at ../Models/Best_BERT_model2. This SavedModel was exported with `tf.saved_model.save`, and lacks the Keras metadata file. Please save your Keras model by calling `model.save` or `tf.keras.models.save_model`. Note that you can still load this SavedModel with `tf.saved_model.load`.

In [ ]:
model.save("../Models/Best_BERT_model_v3", save_format="tf")

INFO:tensorflow:Assets written to: ../Models/Best_BERT_model_v3\assets


INFO:tensorflow:Assets written to: ../Models/Best_BERT_model_v3\assets


In [ ]:
model3 = tf.keras.models.load_model("../Models/Best_BERT_model_v3", compile=False)